In [1]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PandasDataFrameOutputParser



In [ ]:
system_template = "You are a helpful chat assistant:"

model = ChatOpenAI(temperature=0,response_format={"type": "json_object"})



prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{question}")]
)

parser = StrOutputParser()



In [ ]:
chain = prompt_template | model | parser

In [ ]:
data=chain.invoke({"question": "give me the reneue numbers of apple for last 5 years as json. \
                   Use appropriate titles like revenue and year\
                   lint the json before giving it to me\
                   The output should be a valid json object that I could use as it is. There should be no special characters in it\
                   Don't use  new line '\n' characters while printing the json"})

In [ ]:
data=chain.invoke({"question": "give me the reneue numbers of apple for last 5 years as json object"})

In [ ]:
llm_output=data

In [ ]:
llm_output

In [ ]:
import json

cleaned_output = llm_output.strip('()').replace('\n', '').replace('    ', '')

# Parse the JSON data
try:
    data = json.loads(cleaned_output)
    
    # Pretty print the JSON data
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

In [ ]:
print(pretty_json)


In [ ]:
llm_output=pretty_json

In [2]:
llm_output = """
{
    "Apple": {
        "Revenue": {
            "2019": 260.17,
            "2020": 274.52,
            "2021": 365.82,
            "2022": 394.33,
            "2023": 394.33
        },
        "Profit": {
            "2019": 55.26,
            "2020": 57.41,
            "2021": 94.68,
            "2022": 99.8,
            "2023": 101.22
        }
    }
}
"""

In [5]:
import pandas as pd
# Define your desired Pandas DataFrame.
df = pd.DataFrame(
    {
    "Apple": {
        "Revenue": {
            "2019": 260.17,
            "2020": 274.52,
            "2021": 365.82,
            "2022": 394.33,
            "2023": 394.33
        },
        "Profit": {
            "2019": 55.26,
            "2020": 57.41,
            "2021": 94.68,
            "2022": 99.8,
            "2023": 101.22
        }
    }
}
)

# Set up a parser + inject instructions into the prompt template.
parser = PandasDataFrameOutputParser(dataframe=df)

In [11]:
df

,Apple
Revenue,"{'2019': 260.17, '2020': 274.52, '2021': 365.8..."
Profit,"{'2019': 55.26, '2020': 57.41, '2021': 94.68, ..."


In [9]:
# Here's an example of a column operation being performed.
# df_query = "Retrieve the num_wings column."


df_query = "What is average of revenue column."


In [10]:
from langchain_core.prompts import PromptTemplate
model = ChatOpenAI(temperature=0)
# Set up the prompt.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})


OutputParserException: Invalid column: revenue. Please check the format instructions.

In [12]:
llm_output = """
{
    "Apple": {
        "Revenue": {
            "2019": 260.17,
            "2020": 274.52,
            "2021": 365.82,
            "2022": 394.33,
            "2023": 394.33
        },
        "Profit": {
            "2019": 55.26,
            "2020": 57.41,
            "2021": 94.68,
            "2022": 99.8,
            "2023": 101.22
        }
    }
}
"""

In [ ]:
import pprint
pprint.pp(llm_output)

In [13]:
import json

jsondata = json.loads(llm_output)


In [ ]:
import pandas as pd


In [14]:
normalised_data = pd.json_normalize(jsondata['Apple'])

In [15]:
normalised_data

,Revenue.2019,Revenue.2020,Revenue.2021,Revenue.2022,Revenue.2023,Profit.2019,Profit.2020,Profit.2021,Profit.2022,Profit.2023
0,260.17,274.52,365.82,394.33,394.33,55.26,57.41,94.68,99.8,101.22


In [ ]:
import pandas as pd

revenue_df = pd.DataFrame.from_dict(jsondata["Apple"]["Revenue"], orient='index', columns=['Revenue'])


In [ ]:
import json
data = json.loads(llm_output)

# Step 3: Convert the data into a Pandas DataFrame
revenue_df = pd.DataFrame.from_dict(data["Apple"]["Revenue"], orient='index', columns=['Revenue'])
profit_df = pd.DataFrame.from_dict(data["Apple"]["Profit"], orient='index', columns=['Profit'])

# Optionally, you can combine the data into a single DataFrame
combined_df = pd.concat([revenue_df, profit_df], axis=1)

# Display the DataFrame
print(combined_df)

In [ ]:
from langchain.output_parsers import PandasDataFrameOutputParser

# Define your desired Pandas DataFrame.
df = combined_df
# Set up a parser + inject instructions into the prompt template.
parser = PandasDataFrameOutputParser(dataframe=df)

In [ ]:
df

In [ ]:
# Solely for documentation purposes.
from typing import Any, Dict

def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [ ]:
# Solely for documentation purposes.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [ ]:
from langchain_core.prompts import PromptTemplate
model = ChatOpenAI(temperature=0)

# Here's an example of a column operation being performed.
df_query = "Retrieve Revenue."

# Set up the prompt.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})


In [ ]:
parser_output